<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mplfinance


     |████████████████████████████████| 71kB 4.4MB/s 


In [ ]:
pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense
import os


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
sectorname = 'Utilities'
stockname = ['AWK','D','DUK','ES','NEE','PEG','SO','SRE','WEC','XEL']

In [ ]:
for s in stockname:
  pathname = (f'/content/gdrive/My Drive/Colab Notebooks/dataset/Sectors/{sectorname}/{s}.csv')
  df = pd.read_csv (pathname)
  filename = os.path.basename(pathname)
  filename_grab = os.path.splitext(filename)[0]

  df = df[['Adj Close']]

  # A variable for predicting 'forecast_out' days out in future
  for i in [1,2,3,4,5,10,15,30]:
    forecast_out = i
    # Create another column (dependent variable) shifted 'forecast_out' units up
    df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)

    # Create the independent dataset
    # Convert the dataframe to numpy array
    X = np.array(df.drop(['Prediction'],1))
    # Remove the last 'forecast_out' rows
    X = X[:-forecast_out]
    #print(X)
    y = np.array(df['Prediction'])
    # Get all of the y values except the last 'forecast_out' rows
    y = y[:-forecast_out] 

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

    ann_relu = Sequential()
    ann_relu.add(Dense(200, input_dim=1, activation='relu'))
    ann_relu.add(Dense(200, input_dim=200, activation='relu'))
    ann_relu.add(Dense(200, input_dim=200, activation='relu'))
    ann_relu.add(Dense(1, activation='linear'))

    keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
    ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

    ann_relu.summary()

    history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

    y_pred_ann = ann_relu.predict(x_test)

    mse_ann = mean_squared_error(y_test, y_pred_ann)
    print(f'The MSE for the ANN algorithm is: {mse_ann}')

    rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
    print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

    mae_ann = mean_absolute_error(y_test, y_pred_ann)
    print(f'The MAE for the ANN algorithm is: {mae_ann}')

    mape_ann = mean_absolute_percentage_error(y_test, y_pred_ann)
    print(f'The MAPE for the ANN algorithm is: {mape_ann}')

    n_array = np.array([])
    new_array = np.append(n_array, [mse_ann, rmse_ann, mae_ann, mape_ann])
    print(new_array)

    new_df = pd.DataFrame(new_array)

    filepath = f'/content/gdrive/My Drive/Colab Notebooks/results/Sectors/{sectorname}/{forecast_out}Days/{filename_grab}_ann_results.xlsx'
    new_df.to_excel(filepath, index=False)

    keras.backend.clear_session()


61/61 [==============================] - 0s 7ms/step - loss: 4.6930 - mean_absolute_percentage_error: 3.0118 - val_loss: 40.5083 - val_mean_absolute_percentage_error: 11.5446
Epoch 56/200
61/61 [==============================] - 0s 6ms/step - loss: 7.6370 - mean_absolute_percentage_error: 3.4287 - val_loss: 4.0134 - val_mean_absolute_percentage_error: 2.3383
Epoch 57/200
61/61 [==============================] - 0s 5ms/step - loss: 6.6155 - mean_absolute_percentage_error: 3.2699 - val_loss: 3.4990 - val_mean_absolute_percentage_error: 1.8768
Epoch 58/200
61/61 [==============================] - 0s 5ms/step - loss: 6.3157 - mean_absolute_percentage_error: 3.3295 - val_loss: 10.5392 - val_mean_absolute_percentage_error: 4.8460
Epoch 59/200
61/61 [==============================] - 0s 6ms/step - loss: 6.9861 - mean_absolute_percentage_error: 3.5012 - val_loss: 4.4835 - val_mean_absolute_percentage_error: 2.6641
Epoch 60/200
61/61 [==============================] - 0s 7ms/step - loss: 4.9869

In [ ]:
# import pandas as pd
# xyz = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/Sectors/Utilities/AWK.csv') 
# print(xyz.head(1))

In [ ]:
# # Get the stock data
# #df = quandl.get("WIKI/FB")
# #df = pd.read_csv (r'dataset/FB.csv')
# # Take a look at data
# #print(df.head())

# pathname = r'AML.L.csv'

# from google.colab import files
# uploaded = files.upload()
# import io
# df = pd.read_csv(io.BytesIO(uploaded[pathname]))
# df.head()

# #print(df2.head())
# # Get the Adjusted Close
# df = df[['Adj Close']]
# # Take a look at new data
# #print(df.head())

In [ ]:
# filename = os.path.basename(pathname)
# filename_grab = os.path.splitext(filename)[0]

In [ ]:
# # A variable for predicting 'forecast_out' days out in future
# forecast_out = 5

In [ ]:
# # Create another column (dependent variable) shifted 'forecast_out' units up
# df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
# # print(df.tail())

In [ ]:
# # Create the independent dataset
# # Convert the dataframe to numpy array
# X = np.array(df.drop(['Prediction'],1))
# # Remove the last 'forecast_out' rows
# X = X[:-forecast_out]
# #print(X)

In [ ]:
# # Create the dependent dataset
# # Convert the dataframe to numpy array (All of the values including the NaNs)
# y = np.array(df['Prediction'])
# # Get all of the y values except the last 'forecast_out' rows
# y = y[:-forecast_out]
# #print(y)

In [ ]:
# # Split the data into 80% training and 20% testing
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# # print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# # train_set = x_train + y_train
# # test_set = x_test + y_test

In [ ]:
# #Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
# x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
# #print(x_forecast)

In [ ]:
# ann_relu = Sequential()
# ann_relu.add(Dense(200, input_dim=1, activation='relu'))
# ann_relu.add(Dense(200, input_dim=200, activation='relu'))
# ann_relu.add(Dense(200, input_dim=200, activation='relu'))
# ann_relu.add(Dense(1, activation='linear'))

In [ ]:
# keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
# ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [ ]:
# ann_relu.summary()

In [ ]:
# history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

In [ ]:
# y_pred_ann = ann_relu.predict(x_test)

In [ ]:
# mse_ann = mean_squared_error(y_test, y_pred_ann)
# print(f'The MSE for the ANN algorithm is: {mse_ann}')

In [ ]:
# rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
# print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

In [ ]:
# mae_ann = mean_absolute_error(y_test, y_pred_ann)
# print(f'The MAE for the ANN algorithm is: {mae_ann}')

In [ ]:
# mape_ann = mean_absolute_percentage_error(y_test, y_pred_ann)
# print(f'The MAPE for the ANN algorithm is: {mape_ann}')

In [ ]:
# n_array = np.array([])
# new_array = np.append(n_array, [mse_ann, rmse_ann, mae_ann, mape_ann])
# print(new_array)

In [ ]:
# new_df = pd.DataFrame(new_array)

In [ ]:
# filepath = f'{filename_grab}_ann_results.xlsx'
# new_df.to_excel(filepath, index=False)

In [ ]:
# keras.backend.clear_session()